# SQL Query Validation – Weekly Visitor Growth
### SQL Query 3 – Using DuckDB

This notebook validates **Query 3** by loading the SQL logic from the plain text file `query_3.txt` and executing it using DuckDB over the source CSVs.

---

### 🎯 Objective

To analyze restaurant traffic trends over time, this query calculates the **week-over-week growth in visitor volume** based on reservation data.

Specifically:
- Aggregates **total daily reservations**, then rolls them up by week.
- Only includes **full weeks (7 distinct days)** to avoid partial bias.
- Computes the **percentage growth** compared to the previous week using `LAG()`.

---

### 🧠 How it works

1. **`daily_visitors` CTE**:
   - Aggregates total visitors for each `visit_date`.

2. **`weekly_visitors` CTE**:
   - Groups by `week_start`, filters for weeks with exactly 7 days (`HAVING day_count = 7`).

3. **`growth_calculation` CTE**:
   - Uses a window function (`LAG`) to get previous week's value.
   - Computes **absolute and percentage growth** week-over-week.

---

### 🗂️ Output

The final result includes:
- `week_start`
- `total_visitors`
- `prev_week_visitors`
- `growth_pct` (rounded to 2 decimals)

This helps identify whether restaurant activity is increasing or declining on a weekly basis.

You can find the SQL logic in [`sql/query_3.txt`](../sql/query_3.txt).

In [4]:
import duckdb, pandas as pd, pathlib

DATA_PATH = pathlib.Path("../data/raw/Data set")
QUERY_PATH = pathlib.Path("../sql/query_3.txt")

# Connect to DuckDB
con = duckdb.connect()

# Load and clean data
visitors = pd.read_csv(DATA_PATH / 'restaurants_visitors.csv')
visitors = visitors[visitors['visit_date'] != '#VALUE!']
visitors['visit_date'] = pd.to_datetime(visitors['visit_date'])

# Register table
con.register("restaurants_visitors", visitors)

# Load and execute query
sql_query = QUERY_PATH.read_text()
result = con.execute(sql_query).df()
result


,week_start,total_visitors,prev_week_visitors,growth_pct
0,2017-05-15,78.0,170.0,-54.12
1,2017-05-08,170.0,130.0,30.77
2,2017-05-01,130.0,618.0,-78.96
3,2017-04-24,618.0,1469.0,-57.93


In [ ]:
Visitor volume has been declining sharply in three of the last four complete weeks. Only the week of 8 May 2017 showed a rebound (+30.77 %).The most recent week (15 May 2017) fell another 54 %, signalling a potential down-trend that warrants investigation (seasonality, end of Golden-Week holidays, etc.).